In [1]:
with open("../Figures/COVID/AA_sequence_combined.txt") as f:
    sarscov2_combined = f.read().strip()

In [2]:
from pepdata import iedb

In [3]:
df_mhc = iedb.mhc.load_dataframe()

In [4]:
df_tcell = iedb.tcell.load_dataframe()

                      Reference                                     \
                      Assay IRI                      Reference IRI   
0  http://www.iedb.org/assay/29  http://www.iedb.org/reference/274   
1  http://www.iedb.org/assay/37  http://www.iedb.org/reference/281   
2  http://www.iedb.org/assay/42  http://www.iedb.org/reference/283   
3  http://www.iedb.org/assay/43  http://www.iedb.org/reference/283   
4  http://www.iedb.org/assay/47  http://www.iedb.org/reference/275   

                                                                              \
         Type   PubMed ID                                            Authors   
0  Literature  15448372.0  Yi-Hsiang Huang; Mi-Hua Tao; Cheng-po Hu; Wan-...   
1  Literature   7684468.0  V J La Posta; D D Auperin; R Kamin-Lewis; G A ...   
2  Literature  15634878.0    Evan R Jellison; Sung-Kwon Kim; Raymond M Welsh   
3  Literature  15634878.0    Evan R Jellison; Sung-Kwon Kim; Raymond M Welsh   
4  Literature  15519708.0  Ol

In [15]:
mhc_organisms = set(df_mhc["Epitope"]["Organism Name"].unique())

In [16]:
tcell_organisms = set(df_tcell["Epitope"]["Organism Name"].unique())

In [29]:
for o in mhc_organisms:
    if type(o) is str and "SARS" in o:
        print(o)
    if type(o) is str and "respiratory syndrome" in o.lower().replace("-", " "):
        print(">", o)

SARS coronavirus Tor2
Bat SARS CoV Rp3/2004
SARS-related coronavirus
SARS coronavirus Urbani
> Middle East respiratory syndrome-related coronavirus
> Porcine reproductive and respiratory syndrome virus
SARS coronavirus TJF
SARS coronavirus
SARS coronavirus BJ01


In [30]:
for o in tcell_organisms:
    if type(o) is str and "SARS" in o:
        print(o)
    if type(o) is str and "respiratory syndrome" in o.lower().replace("-", " "):
        print(">", o)

SARS coronavirus Tor2
SARS coronavirus Frankfurt 1
SARS coronavirus Urbani
> Middle East respiratory syndrome-related coronavirus
> Porcine reproductive and respiratory syndrome virus
> Porcine reproductive and respiratory syndrome virus HuN4
SARS coronavirus TJF
> Porcine reproductive and respiratory syndrome virus HuN4-F112
SARS coronavirus
SARS coronavirus BJ01


In [32]:
sars_organisms = { 
    "SARS-related coronavirus",
    "SARS coronavirus Urbani",
    "SARS coronavirus Tor2",
    "SARS coronavirus Frankfurt 1",
    "SARS coronavirus TJF",
    "SARS coronavirus",
    "SARS coronavirus BJ01",
}

In [35]:
df_tcell_sars = df_tcell[df_tcell["Epitope"]["Organism Name"].isin(sars_organisms)];
len(df_tcell_sars)

1440

In [36]:
df_mhc_sars = df_mhc[df_mhc["Epitope"]["Organism Name"].isin(sars_organisms)];
len(df_mhc_sars)

13432

In [37]:
sars2_kmers = set([])
for k in {8,9,10,11,12,13,14,15,16,17,18,19,20,21}:
    for i in range(len(sarscov2_combined) - k + 1):
        sars2_kmers.add(sarscov2_combined[i:i+k])

In [38]:
len(sars2_kmers)

135625

In [41]:
df_tcell_sars_in_sars2 = df_tcell_sars[df_tcell_sars["Epitope"]["Description"].isin(sars2_kmers)];
len(df_tcell_sars_in_sars2)

339

In [42]:
df_mhc_sars_in_sars2 = df_mhc_sars[df_mhc_sars["Epitope"]["Description"].isin(sars2_kmers)];
len(df_mhc_sars_in_sars2)

5068